In [1]:
!apt-get update
!apt-get install -y xvfb python-opengl ffmpeg
!pip install gymnasium --upgrade
!pip install pyvirtualdisplay
!pip install JSAnimation

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 3,622 B in 2s (2,217 B/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-2bu

In [2]:
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.animation as animation
import gymnasium as gym


列出所有环境

In [3]:
space_names = ['观测空间', '动作空间', '奖励范围', '最大步数']
df = pd.DataFrame(columns=space_names)

for env_id in gym.envs.registry:
    try:
        env = gym.make(env_id)
        observation_space = env.observation_space
        action_space = env.action_space
        reward_range = env.reward_range
        max_episode_steps = None
        if isinstance(env, gym.wrappers.time_limit.TimeLimit):
            max_episode_steps = env._max_episode_steps
        df.loc[env_id] = [observation_space, action_space, reward_range, max_episode_steps]
    except:
        pass

with pd.option_context('display.max_rows', None):
    display(df)

/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment CartPoleJax-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment Reacher-v2 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment Pusher-v2 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment InvertedPendulum-v2 is out of date. You should consider upgrading to version `v4`.
  logge

,观测空间,动作空间,奖励范围,最大步数
CartPole-v0,Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e...,Discrete(2),"(-inf, inf)",200
CartPole-v1,Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e...,Discrete(2),"(-inf, inf)",500
MountainCar-v0,"Box([-1.2 -0.07], [0.6 0.07], (2,), float32)",Discrete(3),"(-inf, inf)",200
MountainCarContinuous-v0,"Box([-1.2 -0.07], [0.6 0.07], (2,), float32)","Box(-1.0, 1.0, (1,), float32)","(-inf, inf)",999
Pendulum-v1,"Box([-1. -1. -8.], [1. 1. 8.], (3,), float32)","Box(-2.0, 2.0, (1,), float32)","(-inf, inf)",200
Acrobot-v1,Box([ -1. -1. -1. -1. ...,Discrete(3),"(-inf, inf)",500
CartPoleJax-v0,"Box(-inf, inf, (4,), float32)",Discrete(2),"(-inf, inf)",200
CartPoleJax-v1,"Box(-inf, inf, (4,), float32)",Discrete(2),"(-inf, inf)",500
PendulumJax-v0,"Box(-inf, inf, (3,), float32)","Box(-2.0, 2.0, (1,), float32)","(-inf, inf)",200
Blackjack-v1,"(Discrete(32), Discrete(11), Discrete(2))",Discrete(2),"(-inf, inf)",None


In [4]:
from gym.wrappers import RecordVideo
from JSAnimation.IPython_display import display_animation
from IPython.display import display
from pyvirtualdisplay import Display
from IPython.display import HTML

In [5]:
# 设置虚拟显示器
display = Display(visible=0, size=(1400, 900))
display.start()

# 定义一个函数，将Gym的环境渲染为动画
def wrap_env(env):
  env = RecordVideo(env, './video')
  return env

# 在Colab中播放动画
def show_video():
  mp4list = !ls ./video/*.mp4
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = open(mp4,'rb').read()
    display(HTML(f'<video alt="test" src="data:video/mp4;base64,{b64encode(video).decode()}" autoplay=1 loop=1>'))
  else: 
    print("Could not find video")

In [6]:
# 示例代码：在MountainCar-v0环境中使用渲染功能

env = gym.make('MountainCar-v0')
print('观测空间 = {}'.format(env.observation_space))
print('动作空间 = {}'.format(env.action_space))
print('观测范围 = {} ~ {}'.format(env.observation_space.low,
        env.observation_space.high))
print('动作数 = {}'.format(env.action_space.n))

观测空间 = Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
动作空间 = Discrete(3)
观测范围 = [-1.2  -0.07] ~ [0.6  0.07]
动作数 = 3


In [7]:
env = wrap_env(env) # 包装环境以进行渲染
observation = env.reset()

/usr/local/lib/python3.9/dist-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /content/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/monitoring/video_recorder.py:59: UserWarning: WARN: Disabling video recorder because environment <TimeLimit<OrderEnforcing<PassiveEnvChecker<MountainCarEnv<MountainCar-v0>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(
/usr/local/lib/python3.9/dist-packages/gymnasium/envs/classic_control/mountain_car.py:173: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("MountainCar-v0", render_mode="rgb_array")
  gym.logger.warn(


In [8]:
class BespokeAgent:
    def __init__(self, env):
        pass

    def decide(self, observation): # 决策
        position, velocity = observation
        lb = min(-0.09 * (position + 0.25) ** 2 + 0.03, 0.3 * (position + 0.9) ** 4 - 0.008)
        ub = -0.07 * (position + 0.38) ** 2 + 0.07
        if lb < velocity < ub:
            action = 2
        else:
            action = 0
        return action # 返回动作

    def learn(self, *args): # 学习
        pass

agent = BespokeAgent(env)

In [9]:
def play_montecarlo(env, agent, render=False, train=False):
    episode_reward = 0. # 记录回合总奖励，初始化为0
    observation = env.reset() # 重置游戏环境，开始新回合
    while True: # 不断循环，直到回合结束
        if render: # 判断是否显示
            env.render() # 显示图形界面，图形界面可以用 env.close() 语句关闭
        action = agent.decide(observation)
        next_observation, reward, done, info = env.step(action) # 执行动作
        episode_reward += reward # 收集回合奖励
        if train: # 判断是否训练智能体
            agent.learn(observation, action, reward, done) # 学习
        if done: # 回合结束，跳出循环
            break
        observation = next_observation
    return episode_reward # 返回回合总奖励

In [10]:
episode_reward = play_montecarlo(env, agent, render=True)
print('回合奖励 = {}'.format(episode_reward))

ValueError: ignored

ChatGPT
There seem to be two separate code snippets in the question, so I will address them one by one:

Code Snippet 1: The error message "ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()" indicates that the boolean expression inside the if statement (lb < velocity < ub) is producing an array instead of a single boolean value, because the comparison operators are applied element-wise to the arrays.

The cause of this issue is that the input observation is likely a numpy array, and the mathematical operations performed in the lb and ub calculations are producing numpy arrays as output. This means that lb and ub are arrays instead of scalars, which causes the element-wise comparison to produce an array.

To fix this issue, we need to ensure that the lb and ub values are converted to scalars before they are used in the if statement. One way to achieve this is to use the numpy squeeze function to remove any singleton dimensions from the arrays:

In [ ]:
env.close() # 此语句可关闭图形界面
show_video() # 显示动画

In [ ]:
episode_rewards = [play_montecarlo(env, agent) for _ in range(100)]
print('平均回合奖励 = {}'.format(np.mean(episode_rewards)))